# RQ 1: rivers

What is the spatial extent of influence of the river contribution to surface waters in the CAA?
- Does the RCD span full straits in the CAA? If so, which and during all seasons or only for a portion of the year?
- How long does it take for river contributions in the central CAA to reach Baffin Bay?

Relative importance of rivers for trace metal cycles; general component comparison (how important are the rivers for Mn)

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import netCDF4 as nc
import os
from itertools import compress
import datetime as dt

%matplotlib notebook

In [ ]:
import sys
sys.path.insert(0, '/ocean/brogalla/GEOTRACES/analysis-brogalla/modules')
import ariane as ar

##### Load domain dimensions for reference:

In [3]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
cond = (tmask[0,:,:,:] > 0.1) 
land_mask = np.ma.masked_where(cond, tmask[0,:,:,:]) 
tmask = tmask[0,:,1480:2050,160:800]

In [4]:
ref   = nc.Dataset('/data/brogalla/ANHA12/2015/ANHA12-EXH006_y2015m01d05_gridT.nc',  'r')
tlons = np.array(ref.variables['nav_lon'])
tlats = np.array(ref.variables['nav_lat'])
lons  = np.array(ref.variables['nav_lon'])[1480:2050,160:800]
lats  = np.array(ref.variables['nav_lat'])[1480:2050,160:800]
depth = np.array(ref.variables['deptht'])

### Load Mn model run results:

In [ ]:
def load_results(folder_ref, folder_riv, folder_sed, folder_bio, folder_atm, folder_ice, folder_scav):
    
    file   = 'ANHA12_EXH006_2002_monthly.nc'
    ref_monthly  = nc.Dataset(folder_ref+file)
    riv_monthly  = nc.Dataset(folder_riv+file)
    sed_monthly  = nc.Dataset(folder_sed+file)
    bio_monthly  = nc.Dataset(folder_bio+file)
    atm_monthly  = nc.Dataset(folder_atm+file)
    ice_monthly  = nc.Dataset(folder_ice+file)
    scav_monthly = nc.Dataset(folder_scav+file)
    
    dmn_ref_monthly  = np.array(ref_monthly.variables['dissolmn'])[:,0,:,:,:]
    dmn_riv_monthly  = np.array(riv_monthly.variables['dissolmn'])[:,0,:,:,:]
    dmn_sed_monthly  = np.array(sed_monthly.variables['dissolmn'])[:,0,:,:,:]
    dmn_bio_monthly  = np.array(bio_monthly.variables['dissolmn'])[:,0,:,:,:]
    dmn_atm_monthly  = np.array(atm_monthly.variables['dissolmn'])[:,0,:,:,:]
    dmn_ice_monthly  = np.array(ice_monthly.variables['dissolmn'])[:,0,:,:,:]
    dmn_scav_monthly = np.array(scav_monthly.variables['dissolmn'])[:,0,:,:,:]
    
    tmask_full    = np.empty(dmn_ref_monthly.shape)
    tmask_full[:] = tmask
    
    dmn_mref  = np.ma.masked_where((tmask_full < 0.1), dmn_ref_monthly)
    dmn_mriv  = np.ma.masked_where((tmask_full < 0.1), dmn_riv_monthly)
    dmn_msed  = np.ma.masked_where((tmask_full < 0.1), dmn_sed_monthly)
    dmn_mbio  = np.ma.masked_where((tmask_full < 0.1), dmn_bio_monthly)
    dmn_matm  = np.ma.masked_where((tmask_full < 0.1), dmn_atm_monthly)
    dmn_mice  = np.ma.masked_where((tmask_full < 0.1), dmn_ice_monthly)
    dmn_mscav = np.ma.masked_where((tmask_full < 0.1), dmn_scav_monthly)

    return dmn_mref, dmn_mriv, dmn_msed, dmn_mbio, dmn_matm, dmn_mice, dmn_mscav

In [5]:
folder_ref  = '/data/brogalla/run_storage/'
folder_riv  = '/data/brogalla/run_storage/'
folder_sed  = '/data/brogalla/run_storage/'
folder_bio  = '/data/brogalla/run_storage/'
folder_atm  = '/data/brogalla/run_storage/'
folder_ice  = '/data/brogalla/run_storage/'
folder_scav = '/data/brogalla/run_storage/'

In [ ]:
dmn_mref, dmn_mriv, dmn_msed, dmn_mbio, dmn_matm, dmn_mice, dmn_mscav = \
        load_results(folder_ref, folder_riv, folder_sed, folder_bio, folder_atm, folder_ice, folder_scav)

Calculate the proportional contribution of each component to each grid cell:

In [12]:
def calculate_prop(dmn_on, dmn_off):
    # Input arguments:
    # - dmn_on (t,z,y,x) --- concentration of dMn with component on
    # - dmn_off(t,z,y,x) --- concentration of dMn with component off
    
    # Contribution of component at each point
    diff = dmn_on - dmn_off
    
    # Contribution of rivers / total concentration (with rivers on):
    # should be an elementwise operation
    prop  = np.divide(np.abs(diff), dmn_on)
    
    # Output:
    # - prop(t,z,y,x)  --- proportional contribution of component to simulated [dMn]
    return prop

In [ ]:
def all_props(ref, riv, sed, bio, atm, ice, scav):
    
    riv_prop  = calculate_prop(ref, riv)
    sed_prop  = calculate_prop(ref, sed)
    bio_prop  = calculate_prop(ref, bio)
    atm_prop  = calculate_prop(ref, atm)
    ice_prop  = calculate_prop(ref, ice)
    scav_prop = calculate_prop(ref, scav)
    
    return riv_prop, sed_prop, bio_prop, atm_prop, ice_prop, scav_prop

In [ ]:
riv_prop, sed_prop, bio_prop, atm_prop, ice_prop, scav_prop = \
    all_props(dmn_mref, dmn_mriv, dmn_msed, dmn_mbio, dmn_matm, dmn_mice, dmn_mscav)

### Bar plots

In [ ]:
width = 0.5

colors = ['#ccb598', '#007378', '#739f78', '#cc7722', '#cee7fd'] # dust, rivers, scavenging, sediment diffusion

months = ['Jan','Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']

In [ ]:
fig, ax = plt.subplots()

for i in range(0,11):
    ax.bar(months[i], dust_sum_ave[i], width, color=colors[0])
    ax.bar(months[i], rivers_sum_ave[i], width, bottom=dust_sum_ave[i], color=colors[1])
    ax.bar(months[i], scav_sum_ave[i], width, bottom=np.add(dust_sum_ave[i],rivers_sum_ave[i]), color=colors[2])
    ax.bar(months[i], sed_sum_ave[i], width, bottom=np.add(np.add(dust_sum_ave[i], rivers_sum_ave[i]),scav_sum_ave[i]), color=colors[3])
    ax.bar(months[i], ice_sum_ave[i], width, bottom=np.add(np.add(np.add(dust_sum_ave[i], rivers_sum_ave[i]), \
                                                           scav_sum_ave[i]),sed_sum_ave[i]), color=colors[4])
# for legend:
ax.bar(months[0], dust_sum_ave[0]  , 0, color=colors[0], label='Dust')
ax.bar(months[0], rivers_sum_ave[0], 0, color=colors[1], label='Rivers')
ax.bar(months[0], scav_sum_ave[0]  , 0, color=colors[2], label='Scavenging')
ax.bar(months[0], sed_sum_ave[0]   , 0, color=colors[3], label='Sediment diffusion')
ax.bar(months[0], ice_sum_ave[0]   , 0, color=colors[4], label='Ice')
ax.legend(frameon=False)
    
xmin, xmax = ax.get_xlim()
ax.set_xlim([xmin, xmax])
ax.plot([xmin, xmax],[0, 0],'-k',linewidth=1)
ax.ticklabel_format(axis='y',scilimits=(-1, 1))

ax.set_title('Average contribution to Mn concentration')

### Overview of component contributions within water masses: 

- Polar mixed layer (0-40 m)
    - PML depth levels = 1 to 17
- Pacific/halocline water (40-300 m)
    - PW depth levels = 17 to 29
- Atlantic water (300-800 m)
    - AW depth levels = 29 to 35
- Deep water (> 800 m)
    - DW depth levels = 35-50
    
Note that if a component has a negative contribution (i.e. removal), I take the absolute value of this contribution. So, what I'm really looking at is the importance of the component in the concentration/distribution of the trace metal.

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(11,8))
proportions_bar(fig, ax[0,0], PML_ice_sum_ave, PML_dust_sum_ave, PML_rivers_sum_ave, \
                PML_scav_sum_ave, PML_sed_sum_ave, 'Polar mixed layer')
proportions_bar(fig, ax[0,1], PW_ice_sum_ave, PW_dust_sum_ave, PW_rivers_sum_ave, \
                PW_scav_sum_ave, PW_sed_sum_ave, 'Pacific/halocline layer')
proportions_bar(fig, ax[1,0], AW_ice_sum_ave, AW_dust_sum_ave, AW_rivers_sum_ave, \
                AW_scav_sum_ave, AW_sed_sum_ave, 'Atlantic layer')
proportions_bar(fig, ax[1,1], DW_ice_sum_ave, DW_dust_sum_ave, DW_rivers_sum_ave, \
                DW_scav_sum_ave, DW_sed_sum_ave, 'Deep layer', legend=True)

fig.text(0.06, 0.5, 'Proportional contribution to monthly average dMn', \
         ha='center', va='center', rotation='vertical', fontsize=14);